In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=bfa91be1d9bf9744cb1539c68b1b70ae56105b5f771dfacc7467332e6bd8ddb5
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
from pyspark import SparkContext  #DÚVIDA:o q é spark context?
from pyspark.sql import SparkSession

In [3]:
sc = SparkContext.getOrCreate()

In [4]:
spark = SparkSession.builder.appName('PySpark DataFrame From RDD').getOrCreate()

In [5]:
#cria rdd com a função parallelize (tabela)
rdd = sc.parallelize([('C',85,76,87,91), ('B',85,76,87,91), ("A", 85,78,96,92), ("A", 92,76,89,96)], 4)


In [6]:
print(type(rdd))


<class 'pyspark.rdd.RDD'>


In [7]:
#cria df a partir de rdd já existente
sub = ['Division','English','Mathematics','Physics','Chemistry']
marks_df = spark.createDataFrame(rdd, schema=sub)

In [8]:
print(type(marks_df))


<class 'pyspark.sql.dataframe.DataFrame'>


In [9]:
marks_df.printSchema()


root
 |-- Division: string (nullable = true)
 |-- English: long (nullable = true)
 |-- Mathematics: long (nullable = true)
 |-- Physics: long (nullable = true)
 |-- Chemistry: long (nullable = true)



In [10]:
marks_df.show()


+--------+-------+-----------+-------+---------+
|Division|English|Mathematics|Physics|Chemistry|
+--------+-------+-----------+-------+---------+
|       C|     85|         76|     87|       91|
|       B|     85|         76|     87|       91|
|       A|     85|         78|     96|       92|
|       A|     92|         76|     89|       96|
+--------+-------+-----------+-------+---------+



In [11]:
spark = SparkSession.builder.appName('PySpark DataFrame From External Files').getOrCreate()

In [12]:
#cria pyspark df a partir de arquivo externo:
csv_file = spark.read.csv('aluguel.csv', sep = ';', inferSchema = True, header = True)

In [13]:
print(type(csv_file))

<class 'pyspark.sql.dataframe.DataFrame'>


In [14]:
csv_file.printSchema()

root
 |-- Tipo: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Quartos: integer (nullable = true)
 |-- Vagas: integer (nullable = true)
 |-- Suites: integer (nullable = true)
 |-- Area: integer (nullable = true)
 |-- Valor: integer (nullable = true)
 |-- Condominio: integer (nullable = true)
 |-- IPTU: integer (nullable = true)



In [15]:
#transforma pyspark df em pandas df
df = csv_file.toPandas()

In [16]:
type(df)

pandas.core.frame.DataFrame

In [17]:
df = spark.read.csv('cereal.csv', sep = ',', inferSchema = True, header = True)

In [18]:
df.printSchema()


root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: integer (nullable = true)
 |-- protein: integer (nullable = true)
 |-- fat: integer (nullable = true)
 |-- sodium: integer (nullable = true)
 |-- fiber: double (nullable = true)
 |-- carbo: double (nullable = true)
 |-- sugars: integer (nullable = true)
 |-- potass: integer (nullable = true)
 |-- vitamins: integer (nullable = true)
 |-- shelf: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- cups: double (nullable = true)
 |-- rating: double (nullable = true)



In [19]:
df.select('name', 'mfr', 'rating').show(10)
#DÚVIDA: ISSO É SQL? ACHO Q É PYSPARK

+--------------------+---+---------+
|                name|mfr|   rating|
+--------------------+---+---------+
|           100% Bran|  N|68.402973|
|   100% Natural Bran|  Q|33.983679|
|            All-Bran|  K|59.425505|
|All-Bran with Ext...|  K|93.704912|
|      Almond Delight|  R|34.384843|
|Apple Cinnamon Ch...|  G|29.509541|
|         Apple Jacks|  K|33.174094|
|             Basic 4|  G|37.038562|
|           Bran Chex|  R|49.120253|
|         Bran Flakes|  P|53.313813|
+--------------------+---+---------+
only showing top 10 rows



In [20]:
df.withColumn("Calories", df['calories'].cast("Integer")).printSchema()
#DÚVIDA: AQUI ELE CRIOU UMA COLUNA NOVA CHAMADA Calories COM BASE NA COLUNA JÁ EXISTENTE calories? SE SIM, PQ A COLUNA calories NÃO APARECE MAIS?

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- Calories: integer (nullable = true)
 |-- protein: integer (nullable = true)
 |-- fat: integer (nullable = true)
 |-- sodium: integer (nullable = true)
 |-- fiber: double (nullable = true)
 |-- carbo: double (nullable = true)
 |-- sugars: integer (nullable = true)
 |-- potass: integer (nullable = true)
 |-- vitamins: integer (nullable = true)
 |-- shelf: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- cups: double (nullable = true)
 |-- rating: double (nullable = true)



In [22]:
df.groupBy("calories").count().show()

+--------+-----+
|calories|count|
+--------+-----+
|     140|    3|
|     120|   10|
|     100|   17|
|     130|    2|
|      50|    3|
|      80|    1|
|     160|    1|
|      70|    2|
|      90|    7|
|     110|   29|
|     150|    2|
+--------+-----+



In [24]:
df.orderBy("calories").show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|         Puffed Rice|  Q|   C|      50|      1|  0|     0|  0.0| 13.0|     0|    15|       0|    3|   0.5| 1.0|60.756112|
|        Puffed Wheat|  Q|   C|      50|      2|  0|     0|  1.0| 10.0|     0|    50|       0|    3|   0.5| 1.0|63.005645|
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|      Shredded 

In [25]:
from pyspark.sql.functions import when

In [27]:
df.select("name", "vitamins", when(df.vitamins >= "25", "rich in vitamins")).show()
#DÚVIDA: ELE CRIA UMA NOVA COLUNA COM O QUE FOI PEDIDO? SE SIM, ESSA COLUNA FICA SALVA OU É SÓ VISUALIZAÇÃO? PRA SALVAR PRECISO ATRELAR A UM DF?

+--------------------+--------+----------------------------------------------------+
|                name|vitamins|CASE WHEN (vitamins >= 25) THEN rich in vitamins END|
+--------------------+--------+----------------------------------------------------+
|           100% Bran|      25|                                    rich in vitamins|
|   100% Natural Bran|       0|                                                null|
|            All-Bran|      25|                                    rich in vitamins|
|All-Bran with Ext...|      25|                                    rich in vitamins|
|      Almond Delight|      25|                                    rich in vitamins|
|Apple Cinnamon Ch...|      25|                                    rich in vitamins|
|         Apple Jacks|      25|                                    rich in vitamins|
|             Basic 4|      25|                                    rich in vitamins|
|           Bran Chex|      25|                                  

In [28]:
df.filter(df.calories == "100").show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|         Corn Flakes|  K|   C|     100|      2|  0|   290|  1.0| 21.0|     2|    35|      25|    1|   1.0| 1.0|45.863324|
|Cream of Wheat (Q...|  N|   H|     100|      3|  0|    80|  1.0| 21.0|     0|    -1|       0|    2|   1.0| 1.0|64.533816|
|Crispy Wheat & Ra...|  G|   C|     100|      2|  1|   140|  2.0| 11.0|    10|   120|      25|    3|   1.0|0.75|36.176196|
|         Double Chex|  R|   C|     100|      2|  0|   190|  1.0| 18.0|     5|    80|      25|    3|   1.0|0.75|44.330856|
| Frosted Mini-Wheats|  K|   C|     100|      3|  0|     0|  3.0| 14.0|     7|   100|      25|    2|   1.0| 0.8|58.345141|
|        Golden 

In [29]:
#filtrar dados que não são nulos
from pyspark.sql.functions import *
df.filter(df.name.isNotNull()).show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinnamon 

In [30]:
#filtrar nulos
df.filter(df.name.isNull()).show()

+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+
|name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|rating|
+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+
+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+

